In [3]:
!pip install pandas
!pip install numpy
!pip install natsort
!pip install pytube
!pip install moviepy
!pip install opencv-python
!pip install transformers
!pip install PIL


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached moviepy-1.0.3-py3-none-any.whl
  Obtaining dependency information for decorator<5.0,>=4.0.2 from https://files.pythonhosted.org/packages/ed/1b/72a1821152d07cf1d8b6fce298aeb06a7eb90f4d6d41acec9861e7cc6df0/decorator-4.4.2-py2.py3-none-any.whl.metadata
  Using cached decorator-4.4.2-py2.py3-none-any.whl.metadata (4.2 kB)
  Obtaining dependency information for proglog<=1.0.0 from https://files.pythonhosted.org/packages/8b/f5/cab5cf6a540c31f5099043de0ae43990fd9cf66f75ecb5e9f254a4e4d4ee/proglog-0.1.10-py3-none-any.whl.metadata
  Using cached proglog-0.1.10-py3-none-any.whl.metadata (639 bytes)
Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/b6/4d/fbe6d89fde59d8107f0a02816c4ac4542a8f9a85559fdf33c68282affcc1/transformers-4.38.2-py3-none-any.whl.metadata
  Using cached transformers-4.38.2-py3-none-any.whl.metadata (130 kB)
  Obtaining dependency information for tokenizers<0.19,>=0.14 from https://files.pythonhosted.org/packages/c1/02/40725eebedea8175918bd59ab80b2174d6ef3b3ef9ac8ec996e84c38d3ca/tokenizers-0.15.2-cp311-none-win_amd64.whl.metadata
  Using cached tokenizers-0.15.2-cp311-none-win_amd64.whl.metadata (6.8 kB)
Using cached transformers-4.38.2-py3-none-any.whl (8.5 MB)
Using cached tokenizers-0.15.2-cp311-none-win_amd64.whl (2.2 MB)


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\transformers-cli.exe' -> 'C:\\Python311\\Scripts\\transformers-cli.exe.deleteme'


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
from natsort import index_natsorted
import json
from pytube import YouTube
from pytube.exceptions import VideoUnavailable
from moviepy.video.io.VideoFileClip import VideoFileClip
from datetime import datetime
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import cv2
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import os

Part 0: Data Pre-processing
1. Get information on the json file

In [4]:
with open("mlda_data.json") as f:
    data = json.load(f)

clip_counts = 0

for i in data.keys(): # video id
    for j in data[i]["clip"].keys(): # clips
        for k in data[i]["clip"][j]["scene_split"]:
            clip_counts += 1

print(len(data))
print(clip_counts)

11239
933677


2. Convert json into csv

In [21]:
df = pd.DataFrame()

video_id = []
video_ext = []
span_start = []
span_end = []
fps = []
scene_cut_start = []
scene_cut_end = []
clip_id = []
caption = []
clip_frames = []

for i in data.keys(): # for each video id
    temp_df = pd.DataFrame()
    for j in data[i]["clip"]: # for each video clips
        temp_end = []
        for k in range(len(data[i]["clip"][j]["scene_split"])):  # for each scenes
            video_id.append(i)
            video_ext.append(j)
            span_start.append("'" + data[i]["clip"][j]["span"][0])
            span_end.append("'" + data[i]["clip"][j]["span"][1])
            fps.append(data[i]["clip"][j]["fps"])
            scene_cut_start.append(data[i]["clip"][j]["scene_split"][k]["scene_cut"][0])
            scene_cut_end.append(data[i]["clip"][j]["scene_split"][k]["scene_cut"][1])
            temp_end.append(data[i]["clip"][j]["scene_split"][k]["scene_cut"][1])
            clip_id.append(data[i]["clip"][j]["scene_split"][k]["clip_id"])
            caption.append(data[i]["clip"][j]["scene_split"][k]["caption"])
        
        frames = max(temp_end)

        for k in range(len(data[i]["clip"][j]["scene_split"])):
            clip_frames.append(frames + 1)
           
df["video_id"] = video_id
df["video_ext"] = video_ext 
df["span_start"] = span_start
df["span_end"] = span_end
df["fps"] = fps
df["scene_cut_start"] = scene_cut_start
df["scene_cut_end"] = scene_cut_end
df["clip_id"] = clip_id
df["caption"] = caption
df["clip_frames"] = clip_frames
df.head(20)

,video_id,video_ext,span_start,span_end,fps,scene_cut_start,scene_cut_end,clip_id,caption,clip_frames
0,gpAkjSy_8iY,gpAkjSy_8iY.0.mp4,'00:00:00.030,'00:00:07.740,30.0,0,-1,gpAkjSy_8iY.0_0,a man in a suit and tie standing next to a new...,-1
1,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,9,37,gpAkjSy_8iY.8_1,a street that has power lines down on it,337
2,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,37,101,gpAkjSy_8iY.8_2,a woman standing in the middle of a flooded st...,337
3,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,101,153,gpAkjSy_8iY.8_3,a group of people standing around a car in a f...,337
4,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,153,337,gpAkjSy_8iY.8_4,bill patterson is interviewed on a newscast,337
5,gpAkjSy_8iY,gpAkjSy_8iY.8.mp4,'00:01:46.479,'00:01:57.009,30.0,0,9,gpAkjSy_8iY.8_0,a picture of a city street with power lines in...,337
6,gpAkjSy_8iY,gpAkjSy_8iY.10.mp4,'00:02:14.739,'00:02:22.380,30.0,204,206,gpAkjSy_8iY.10_1,an airplane flying through the dark sky,206
7,gpAkjSy_8iY,gpAkjSy_8iY.10.mp4,'00:02:14.739,'00:02:22.380,30.0,0,204,gpAkjSy_8iY.10_0,a picture of a man in a suit and tie,206
8,gpAkjSy_8iY,gpAkjSy_8iY.9.mp4,'00:01:57.009,'00:02:14.739,30.0,328,558,gpAkjSy_8iY.9_1,a picture of a man in a suit and tie,558
9,gpAkjSy_8iY,gpAkjSy_8iY.9.mp4,'00:01:57.009,'00:02:14.739,30.0,0,328,gpAkjSy_8iY.9_0,a fire on the left and a photo of the same fir...,558


In [22]:
# using natural sorting to sort the ids (to sort numbers following string)
sorted_indices = index_natsorted(df[['video_ext', 'clip_id']].to_numpy())

# Reorder the DataFrame using the sorted indices
new_df = df.iloc[sorted_indices]

In [23]:
new_df.to_csv("info.csv", index=False) # intermediate save
df = new_df

3. Additional processing to get more information

In [24]:
# Function to convert timestamp to seconds with milliseconds
def timestamp_to_seconds(timestamp):
    hours, minutes, seconds_ms = map(str, timestamp.split(':'))

    if '.' in seconds_ms:
        seconds, milliseconds = map(int, seconds_ms.split('.'))
    else:
        seconds = int(seconds_ms)
        milliseconds = 0

    hours = hours[1:]

    return int(hours) * 3600 + int(minutes) * 60 + seconds + milliseconds / 1000

In [25]:
def find_duration(span_start, span_end):
    start_time = timestamp_to_seconds(span_start)
    end_time = timestamp_to_seconds(span_end)
    return end_time - start_time

In [26]:
# filtering rows where fps is larger than 23 only
df = df[df['fps'] >= 23]

In [27]:
# find duration of each scenes
df['duration'] = df.apply(lambda row: find_duration(row['span_start'], row['span_end']), axis=1)    
df

C:\Users\ACER\AppData\Local\Temp\ipykernel_16264\2253615893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duration'] = df.apply(lambda row: find_duration(row['span_start'], row['span_end']), axis=1)


,video_id,video_ext,span_start,span_end,fps,scene_cut_start,scene_cut_end,clip_id,caption,clip_frames,duration
0,gpAkjSy_8iY,gpAkjSy_8iY.0.mp4,'00:00:00.030,'00:00:07.740,30.00000,0,-1,gpAkjSy_8iY.0_0,a man in a suit and tie standing next to a new...,-1,7.710
34,gpAkjSy_8iY,gpAkjSy_8iY.1.mp4,'00:00:07.740,'00:00:23.789,30.00000,0,-1,gpAkjSy_8iY.1_0,a news anchor standing in front of a fire warn...,-1,16.049
10,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.00000,0,106,gpAkjSy_8iY.2_0,a man in a suit and tie standing next to a new...,419,12.180
11,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.00000,106,133,gpAkjSy_8iY.2_1,a truck in front of a large fire,419,12.180
12,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.00000,133,201,gpAkjSy_8iY.2_2,a firefighter using a hose to put out a fire,419,12.180
...,...,...,...,...,...,...,...,...,...,...,...
933675,msKdwfqKgro,msKdwfqKgro.0.mp4,'00:00:00.000,'00:00:05.520,30.00000,0,-1,msKdwfqKgro.0_0,a man standing in front of two cars in a garage,-1,5.520
933673,msKdwfqKgro,msKdwfqKgro.1.mp4,'00:00:05.520,'00:00:16.500,30.00000,0,-1,msKdwfqKgro.1_0,a man standing in front of two cars in a garage,-1,10.980
933674,msKdwfqKgro,msKdwfqKgro.2.mp4,'00:00:16.500,'00:00:21.710,30.00000,0,-1,msKdwfqKgro.2_0,a man standing in front of a green and a red m...,-1,5.210
933670,mxv7EotS-OU,mxv7EotS-OU.0.mp4,'00:00:02.683,'00:00:14.160,29.97003,0,-1,mxv7EotS-OU.0_0,an image of a man being rescued from the water,-1,11.477


In [28]:
# calculate number of frames of each scenes
df['total_frames'] = (df['duration'] * df['fps']).astype(int)

C:\Users\ACER\AppData\Local\Temp\ipykernel_16264\2693850716.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_frames'] = (df['duration'] * df['fps']).astype(int)


In [29]:
# Remove rows where 'scene_cut_end' column has a value of -1
# long scenes will have many transitions, hence low quality
df = df[df['scene_cut_end'] != -1]

df

,video_id,video_ext,span_start,span_end,fps,scene_cut_start,scene_cut_end,clip_id,caption,clip_frames,duration,total_frames
10,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,0,106,gpAkjSy_8iY.2_0,a man in a suit and tie standing next to a new...,419,12.18,365
11,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,106,133,gpAkjSy_8iY.2_1,a truck in front of a large fire,419,12.18,365
12,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,133,201,gpAkjSy_8iY.2_2,a firefighter using a hose to put out a fire,419,12.18,365
13,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,201,237,gpAkjSy_8iY.2_3,a fire truck and two men standing next to it,419,12.18,365
14,gpAkjSy_8iY,gpAkjSy_8iY.2.mp4,'00:00:23.789,'00:00:35.969,30.0,237,331,gpAkjSy_8iY.2_4,a group of firefighters standing next to a fence,419,12.18,365
...,...,...,...,...,...,...,...,...,...,...,...,...
933629,hnzm4IBXnt8,hnzm4IBXnt8.22.mp4,'00:07:07.430,'00:07:37.240,25.0,705,748,hnzm4IBXnt8.22_14,a man hugging a woman dressed as a princess,849,29.81,745
933631,hnzm4IBXnt8,hnzm4IBXnt8.22.mp4,'00:07:07.430,'00:07:37.240,25.0,748,794,hnzm4IBXnt8.22_15,a group of people dressed in costumes on a stage,849,29.81,745
933634,hnzm4IBXnt8,hnzm4IBXnt8.22.mp4,'00:07:07.430,'00:07:37.240,25.0,794,849,hnzm4IBXnt8.22_16,a sign that says caba boi in russian,849,29.81,745
933667,mqgYOA3KN4c,mqgYOA3KN4c.0.mp4,'00:00:00.000,'00:00:39.650,25.0,0,686,mqgYOA3KN4c.0_0,a person using a pen to draw a picture,992,39.65,991


In [40]:
df['bias'] = df['clip_frames'] - df['total_frames']
df = df[df['bias'] >= 0]
# df.head(20)

C:\Users\ACER\AppData\Local\Temp\ipykernel_16264\1886759273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bias'] = df['clip_frames'] - df['total_frames']


In [46]:
df['actual_scene_cut_start'] = df['scene_cut_start'] - df['bias']
df['actual_scene_cut_end'] = df['scene_cut_end'] - df['bias']

# make sure that start is at least 0
df['actual_scene_cut_start'] = df['actual_scene_cut_start'].apply(lambda x: max(0, x))
df = df[df['actual_scene_cut_end'] > 0]

# number of frames in scene should be at least its fps, and lower than 500
df['scene_frames'] = df['actual_scene_cut_end'] - df['actual_scene_cut_start']
df = df[df['scene_frames'] >= df['fps']]
df = df[df['scene_frames'] <= 500]

In [48]:
df.to_csv("output.csv", index=False) # final of pre-processing

Part 1: Downloading Videos and Preparing Frames

Download videos from YouTube

In [ ]:
# Function to download a YouTube video based on timestamps
def download_video(url, span_start, span_end, video_ext, output_folder, df):
    # Convert timestamp to seconds with milliseconds
    start_time = timestamp_to_seconds(span_start)
    end_time = timestamp_to_seconds(span_end)

    temp_file_path = ""
    final_file_path = ""

    try:
        # Create YouTube object
        yt = YouTube(url)

        # Get the video stream
        video_stream = yt.streams.filter(file_extension="mp4").first()

        # Set up file paths
        temp_file_path = os.path.join(output_folder, video_ext + "_temp.mp4")
        final_file_path = os.path.join(output_folder, video_ext)

        # Download the video
        video_stream.download(filename=temp_file_path)
    except VideoUnavailable:
        print("This video is no longer available")
        df = df.drop(df[df['video_ext'] == video_ext].index)

    try:
        # Subclip the downloaded video using moviepy
        video_clip = VideoFileClip(temp_file_path).subclip(start_time, end_time)
        video_clip.write_videofile(final_file_path)

        # Print downloaded and expected durations for verification
        downloaded_duration = video_clip.duration
        expected_duration = end_time - start_time
        print(f"Downloaded Duration: {downloaded_duration}, Expected Duration: {expected_duration}")

    except Exception as e:
        print(f"Error during subclip creation: {e}")
    finally:
        # Remove the temporary file
        if os.path.exists(temp_file_path):
            os.remove(temp_file_path)

In [ ]:
# Set the output folder path in your Google Drive
output_folder = '/MLDA Videos/clips'

# Keep track of downloaded clips to avoid duplicates
downloaded_clips = set()
checkpoint = 491

# Iterate through all rows in the DataFrame
for count, (index, row) in enumerate(df.iterrows()):
    # Check if the clip has already been downloaded
    clip_identifier = (row['span_start'], row['span_end'])
    if clip_identifier not in downloaded_clips and count >= checkpoint:
        # Download video for each row
        download_video(f"https://youtube.com/watch?v={row['video_id']}", row['span_start'], row['span_end'], row['video_ext'], output_folder, df)
        # download_video(row['url'], row['span_start'], row['span_end'], row['video_ext'], output_folder)

        # Add the clip identifier to the set
        downloaded_clips.add(clip_identifier)
        print("Movies downloaded: " + len(downloaded_clips) + " Count: " + count)

        # Stop after downloading 5 unique clips (modify this to how many number of clips u want to download)
        if count >= 1100:
            break

# Optionally, you can print a message indicating the process is complete
print("Videos downloaded and saved to Google Drive.")

In [ ]:
# after removing unavailable videos
df.to_csv("available_videos.csv", index=False)

Cut clips into scenes

In [ ]:
if 'Unnamed: 0' in df.columns:
    df_cleaned = df.drop('Unnamed: 0', axis=1)
else:
    df_cleaned = df.copy()  # If the column doesn't exist, just make a copy of the DataFrame

df_cleaned['actual_scene_cut_start'] = df_cleaned['actual_scene_cut_start'] + 1
df_cleaned['start_time'] = df_cleaned['actual_scene_cut_start'] / df_cleaned['fps']

df_cleaned['actual_scene_cut_end'] = df_cleaned['actual_scene_cut_end'] - 2
df_cleaned['end_time'] = df_cleaned['actual_scene_cut_end'] / df_cleaned['fps']

df_cleaned['end_minus_start'] = df_cleaned['end_time'] - df_cleaned['start_time']

Extracting frames from scenes

In [ ]:
# Folder path where your original clips are stored
# original_clips_folder = "/content/drive/My Drive/MLDA Hackathon/downloaded_clips_nominusone"
original_clips_folder = "MLDA Videos/clips"

# Folder path where you want to save the cut subclips
output_folder = "MLDA Videos/scenes"

# Update the video_ext column in your DataFrame
df_cleaned['video_ext'] = df_cleaned['video_ext'].apply(lambda x: os.path.join(original_clips_folder, x))

# Function to cut videos and save subclips
def cut_and_save_subclip(row):
    video_path = row['video_ext']
    clip_id = row['clip_id']
    start_time = row['start_time']
    end_time = row['end_time']

    # Check if the video file exists
    if not os.path.exists(video_path):
        print(f"Video file not found: {video_path}")
        return

    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Output path for the subclip
    output_path = os.path.join(output_folder, f"{clip_id}.mp4")

    # Cut the subclip
    ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=output_path)

# Apply the function to each row in the DataFrame
df_cleaned.apply(cut_and_save_subclip, axis=1)

In [ ]:
def extract_frames_from_folder(frames_folder, output_folder):
    # Get a list of all image files in the frames folder
    frame_files = sorted([f for f in os.listdir(frames_folder) if f.endswith(('.jpg'))])

    # Ensure there are frames in the folder
    if not frame_files:
        print(f"No frame images found in the folder {frames_folder}.")
        return

    # Extract first frame
    first_frame = cv2.imread(os.path.join(frames_folder, frame_files[0]))
    cv2.imwrite(os.path.join(output_folder, frame_files[0]), first_frame)

    # Extract median frame
    median_frame_index = len(frame_files) // 2
    median_frame = cv2.imread(os.path.join(frames_folder, frame_files[median_frame_index]))
    cv2.imwrite(os.path.join(output_folder, frame_files[median_frame_index]), median_frame)

    # Extract last frame
    last_frame = cv2.imread(os.path.join(frames_folder, frame_files[-1]))
    cv2.imwrite(os.path.join(output_folder, frame_files[-1]), last_frame)

    print(f"Frames extracted successfully from {frames_folder}.")

def extract_frames_from_multiple_folders(main_folder, output_folder):
    # Loop through all folders in the main folder
    for folder_name in os.listdir(main_folder):
        folder_path = os.path.join(main_folder, folder_name)
        if os.path.isdir(folder_path):
            # Create a subfolder in the output folder for the frames from this folder
            output_subfolder = os.path.join(output_folder, folder_name)
            if not os.path.exists(output_subfolder):
                os.makedirs(output_subfolder)

            # Extract frames from this folder
            extract_frames_from_folder(folder_path, output_subfolder)

In [ ]:
# Define the main folder containing the frame folders
main_folder = 'MLDA Videos/cutframes'  # Replace with the path to your main folder

# Define the output folder where extracted frames will be saved
output_folder = 'MLDA Videos/extracted_frames1'  # Replace with your desired output folder path
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Extract frames from multiple folders
extract_frames_from_multiple_folders(main_folder, output_folder)

Part 2: Score Evaluation

- 2.1 DOVER Score

- 2.2 CLIP Score

In [ ]:
# normalise the data (min -> 0, max -> 1)
def normalise(col):
    col_max = col.max()
    col_min = col.min()
    normalized_data = (col - col_min) / (col_max - col_min)
    return normalized_data

In [ ]:
# if the difference between 3 frame CLIP score is large, means low quality clip
# penalise by using score * (1-diff)
def penalise_with_diff(scores, diff):
    final_scores = (1 - diff) * scores
    return final_scores

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

def calculate_score(image, text):
    # If there's multiple text in the caption separated by ";"
    labels = text.split(";")
    labels = [l.strip() for l in labels]
    labels = list(filter(None, labels))
    if len(labels) == 0:
        return dict()
    
    # Text and image inputs are passed into the text and image encoder respectively and obtain their dot product
    inputs = processor(text=labels, images=image, return_tensors="pt", padding=True)
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image.detach().numpy()

    results_dict = {
        label: score / 100.0 for label, score in zip(labels, logits_per_image[0])
    }

    return results_dict

In [ ]:
def process_image_folder(jpg_path, df):
    # Get the list of items in the directory
    directory_contents = os.listdir(jpg_path)

    df_score = pd.DataFrame(columns=['text', 'first_frame_clip_score', 'middle_frame_clip_score', 'last_frame_clip_score'])
    df_tmp = pd.DataFrame()

    # Filter only directories
    directories = [item for item in directory_contents if os.path.isdir(os.path.join(jpg_path, item))]

    for folder_name in directories:
        matching_rows = df[df['clip_id'].isin([folder_name])]
        if not matching_rows.empty:
            folder_path = os.path.join(jpg_path, folder_name)
            print(f"Processing folder: {folder_path}")
            folder_contents = os.listdir(folder_path)
            jpg_files = [file for file in folder_contents if file.endswith('.jpg')]
            jpg_files.sort()

            # Check if there are any JPG files in the folder
            if jpg_files:
                # first_index = 0
                # middle_index = len(jpg_files) // 2
                # last_index = len(jpg_files) - 1

                first_jpg = jpg_files[0]
                middle_jpg = jpg_files[1]
                last_jpg = jpg_files[2]

                print(f"First, Middle, Last: {first_jpg}, {middle_jpg}, {last_jpg}")

                first_jpg_path = os.path.join(folder_path, first_jpg)
                middle_jpg_path = os.path.join(folder_path, middle_jpg)
                last_jpg_path = os.path.join(folder_path, last_jpg)

                first_image = Image.open(first_jpg_path)
                middle_image = Image.open(middle_jpg_path)
                last_image = Image.open(last_jpg_path)

                caption = matching_rows['caption'].values[0]  # Access the first element of the 'caption' column

                first_frame_clip_score = calculate_score(first_image, caption)
                middle_frame_clip_score = calculate_score(middle_image, caption)
                last_frame_clip_score = calculate_score(last_image, caption)

                df_tmp = pd.DataFrame({'text': list(first_frame_clip_score.keys()), 
                                       'first_frame_clip_score': list(first_frame_clip_score.values()), 
                                       'middle_frame_clip_score': list(middle_frame_clip_score.values()), 
                                       'last_frame_clip_score': list(last_frame_clip_score.values())})
                df_tmp['first_jpg'] = first_jpg
                df_tmp['middle_jpg'] = middle_jpg
                df_tmp['last_jpg'] = last_jpg

                df_score = pd.concat([df_score, df_tmp], ignore_index=True)

    return df_score

jpg_path = r"C:\Users\YongZane\Desktop\MLDA_Hackathon_2024\jpg_frames"
df_score = process_image_folder(jpg_path, df)
df_score.head()

In [ ]:
for index, row in df_score.iterrows():
    clip_score = np.array((row['first_frame_clip_score'], row['middle_frame_clip_score'], row['last_frame_clip_score']))
    clip_score_max_diff = np.max(clip_score) - np.min(clip_score) # find dissimarility of text-image pair in each scene
    avg_clip_score = np.mean(clip_score) # calculate average of 3 frame clip scores

    df_score.at[index, 'clip_score_max_diff'] = clip_score_max_diff
    df_score.at[index, 'avg_clip_score'] = avg_clip_score

normalised_clip = normalise(df_score['avg_clip_score'])
df['clip_score'] = penalise_with_diff(normalised_clip, df_score['clip_score_max_diff'])

df.head()

In [ ]:
df.to_csv('with CLIP score.csv')

- 2.3 Combined Score Evaluation

In [ ]:
df['total_score'] = df['dover_score'] + df['clip_score']

Part 3: Sorting and Ranking

In [ ]:
top_n = 10
df = df.sort_values(ascending=False, by=['total_score'])
df = df[:top_n]

df

In [ ]:
result = df
result.to_csv("result.csv", index=False)

In [ ]:
text_video_pairs = df[['clip_id', 'caption', 'total_score']]
text_video_pairs
text_video_pairs.to_csv("text_video.csv", index=False)